In [76]:
import numpy as np
import pandas as pd
import glob

In [77]:
path = r'C:\Users\movin\data-x-team-project\Tagged Photos' # csv file path
allFiles = glob.glob(path + "/*.csv")
df = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_, index_col=None, header=0)
    list_.append(df)
df = pd.concat(list_, ignore_index=True)

In [78]:
len(df.index)

15997

In [79]:
df.head()

,Unnamed: 0,fileName,Labels,FaceDetails
0,0,Facebook/ENFJ/1000006876806379.jpg,"[{'Name': 'Human', 'Confidence': 99.2957839965...",[{'BoundingBox': {'Width': 0.11999999731779099...
1,1,Facebook/ENFJ/1001091583356351.jpg,"[{'Name': 'Art', 'Confidence': 52.856147766113...",[]
2,2,Facebook/ENFJ/1001328126675771.jpg,"[{'Name': 'People', 'Confidence': 99.012985229...","[{'BoundingBox': {'Width': 0.4146634638309479,..."
3,3,Facebook/ENFJ/10100163853254202.jpg,"[{'Name': 'Art', 'Confidence': 91.370552062988...",[]
4,4,Facebook/ENFJ/10100238407492499.jpg,"[{'Name': 'Human', 'Confidence': 99.3081130981...","[{'BoundingBox': {'Width': 0.6088888645172119,..."


In [80]:
# get the personality for every photo
personality_list = []
for i in range(len(df.index)):
    personality_list.append(df['fileName'][i][9:13])
len(personality_list)

15997

In [81]:
personality_list[-10:]

['ISTP',
 'ISTP',
 'ISTP',
 'ISTP',
 'ISTP',
 'ISTP',
 'ISTP',
 'ISTP',
 'ISTP',
 'ISTP']

In [82]:
# add the personality column to dataframe
se = pd.Series(personality_list)
df['Personality'] = se.values
df.head()

,Unnamed: 0,fileName,Labels,FaceDetails,Personality
0,0,Facebook/ENFJ/1000006876806379.jpg,"[{'Name': 'Human', 'Confidence': 99.2957839965...",[{'BoundingBox': {'Width': 0.11999999731779099...,ENFJ
1,1,Facebook/ENFJ/1001091583356351.jpg,"[{'Name': 'Art', 'Confidence': 52.856147766113...",[],ENFJ
2,2,Facebook/ENFJ/1001328126675771.jpg,"[{'Name': 'People', 'Confidence': 99.012985229...","[{'BoundingBox': {'Width': 0.4146634638309479,...",ENFJ
3,3,Facebook/ENFJ/10100163853254202.jpg,"[{'Name': 'Art', 'Confidence': 91.370552062988...",[],ENFJ
4,4,Facebook/ENFJ/10100238407492499.jpg,"[{'Name': 'Human', 'Confidence': 99.3081130981...","[{'BoundingBox': {'Width': 0.6088888645172119,...",ENFJ


In [83]:
# define a function to get tags for each photo
def get_tag(astring):
    split_list = re.split(': |,',astring)
    taglist = []
    for i in range(0,len(split_list)//4):
        taglist.append(split_list[4*i+1])
    return(taglist)

In [84]:
df['Labels'][0]

"[{'Name': 'Human', 'Confidence': 99.29578399658203}, {'Name': 'People', 'Confidence': 99.29878234863281}, {'Name': 'Person', 'Confidence': 99.29878234863281}, {'Name': 'Apparel', 'Confidence': 75.07823181152344}, {'Name': 'Clothing', 'Confidence': 75.07823181152344}, {'Name': 'Maillot', 'Confidence': 60.394203186035156}, {'Name': 'Female', 'Confidence': 60.05233383178711}, {'Name': 'Dress', 'Confidence': 51.23127746582031}, {'Name': 'Bra', 'Confidence': 50.790958404541016}, {'Name': 'Lingerie', 'Confidence': 50.790958404541016}, {'Name': 'Underwear', 'Confidence': 50.790958404541016}]"

In [85]:
# get all the tags
tagdict = {}
for i in range(len(df.index)):
    tagdict[df['fileName'][i]] = get_tag(df['Labels'][i])
len(tagdict)

15997

In [86]:
# get set of tags
tagset = set()
for key in tagdict:
    for value in tagdict[key]:
        tagset.add(value)
len(tagset) # in total 1797 different tags

1797

In [87]:
taglist = []
for key in tagdict:
    taglist.append(tagdict[key])
len(taglist)

15997

In [88]:
taglist[0]

["'Human'",
 "'People'",
 "'Person'",
 "'Apparel'",
 "'Clothing'",
 "'Maillot'",
 "'Female'",
 "'Dress'",
 "'Bra'",
 "'Lingerie'",
 "'Underwear'"]

In [89]:
# get rid of extra ""
for i in range(len(taglist)):
    for j in range(len(taglist[i])):
        taglist[i][j] = taglist[i][j][1:-1]

In [90]:
taglist[0]

['Human',
 'People',
 'Person',
 'Apparel',
 'Clothing',
 'Maillot',
 'Female',
 'Dress',
 'Bra',
 'Lingerie',
 'Underwear']

In [91]:
# join each word of an element of taglist into a string
for i in range(len(taglist)):
    taglist[i] = ' '.join(word for word in taglist[i])

In [92]:
taglist[0]

'Human People Person Apparel Clothing Maillot Female Dress Bra Lingerie Underwear'

In [93]:
# sentiment classification w/ scikit-learn, feature vector, bag of words model
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics # for confusion matrix, accuracy score etc
from sklearn.model_selection import train_test_split

In [94]:
# split the training set and testing set
x_train, x_test, y_train, y_test = train_test_split(taglist, df['Personality'], random_state=0, test_size=.2)

# CountVectorize
vectorizer = CountVectorizer(analyzer="word", tokenizer=None, preprocessor=None, stop_words=None, max_features=2000)

In [95]:
%%time
# Transform the text data to feature
# Only fit training data (to mimic real world)

vectorizer.fit(x_train)

Wall time: 159 ms


CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=2000, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [96]:
# Check that it worked, 
# now we have fitted a model that can transform features
# to sparse matrix representation

print(vectorizer.get_feature_names()[:10])

['abies', 'abyssinian', 'acanthaceae', 'accessories', 'accipiter', 'activities', 'adapter', 'adorable', 'adventure', 'aerial']


In [97]:
train_bag = vectorizer.transform(x_train) #transform to a feature matrix
test_bag = vectorizer.transform(x_test)

In [98]:
# 15997 photos, 1668 feartures
print(train_bag.toarray().shape)
print(test_bag.toarray().shape)

(12797, 1668)
(3200, 1668)


In [99]:
type(train_bag) # sparse matrix representation

print(train_bag)

  (0, 189)	1
  (0, 805)	1
  (0, 1074)	1
  (0, 1129)	1
  (0, 1137)	1
  (0, 1196)	1
  (0, 1290)	1
  (0, 1320)	1
  (0, 1383)	1
  (1, 374)	1
  (1, 380)	1
  (1, 805)	1
  (1, 827)	1
  (1, 1078)	1
  (1, 1129)	1
  (1, 1137)	1
  (1, 1445)	1
  (1, 1570)	1
  (2, 31)	1
  (2, 32)	1
  (2, 213)	1
  (2, 214)	1
  (2, 409)	1
  (2, 586)	1
  (2, 591)	1
  :	:
  (12795, 132)	1
  (12795, 180)	1
  (12795, 356)	1
  (12795, 586)	1
  (12795, 696)	1
  (12795, 702)	1
  (12795, 739)	1
  (12795, 805)	1
  (12795, 859)	1
  (12795, 1129)	1
  (12795, 1137)	1
  (12795, 1196)	1
  (12795, 1320)	1
  (12795, 1450)	1
  (12796, 12)	1
  (12796, 586)	1
  (12796, 696)	1
  (12796, 702)	1
  (12796, 739)	1
  (12796, 741)	1
  (12796, 805)	1
  (12796, 1129)	1
  (12796, 1137)	1
  (12796, 1320)	1
  (12796, 1450)	1


In [100]:
# classify with Random Forest Model
from sklearn.ensemble import RandomForestClassifier

## Initialize a Random Forest classifier with 50 trees
# hyperparameter n_estimators always set in instantiation

forest = RandomForestClassifier(n_estimators = 50)

In [101]:
%%time
# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the target variable

forest = forest.fit(train_bag, y_train)

Wall time: 16.3 s


In [102]:
# Make predictions

train_predictions = forest.predict(train_bag)
valid_predictions = forest.predict(test_bag)

In [103]:
# training accuracy
metrics.accuracy_score(y_train,train_predictions)

0.6454637805735719

In [104]:
# test accuracy
metrics.accuracy_score(y_test,valid_predictions)

0.049375000000000002

In [105]:
# Confusion matrix
# Is the number of False Positives and True negatives approx 50/50?
metrics.confusion_matrix(y_test,valid_predictions)

array([[ 9, 12, 11, 15, 11, 13, 14, 12, 14,  5, 14, 21, 10, 14, 13, 16],
       [12, 12,  6, 13,  7, 16, 12, 16, 18, 16, 17,  6,  7, 15,  9, 14],
       [15, 11,  6, 14, 10, 17, 16,  5, 10, 14, 12, 17, 14,  7,  6, 12],
       [10, 12, 11,  9, 10,  8,  4, 11, 22, 10,  7, 22, 12, 10,  8, 15],
       [ 7, 10, 17,  6,  9, 17, 19,  9, 13, 12, 11, 16, 17,  8, 12, 12],
       [10, 19, 12, 11, 20, 12, 13, 20,  5, 13, 13, 12,  7, 14, 12, 11],
       [12,  8, 14, 12, 24, 10,  9, 21, 13,  7, 15,  8,  9, 10,  7, 10],
       [16, 19, 17,  8, 16, 22, 17, 10, 10,  9,  8, 19, 16, 15, 10, 13],
       [ 4, 20,  8, 17, 15,  9, 12, 12, 13, 19, 17,  9, 14,  9, 19, 10],
       [13, 11, 10,  9, 11, 11, 13, 12, 18,  7, 17, 17,  9, 13,  7, 13],
       [12, 18, 11,  8, 16, 10, 11,  5, 17, 13, 13, 18, 17,  9,  8, 11],
       [10, 13, 17, 12, 13, 21,  7,  7, 19, 19, 12, 12, 20, 11,  8, 17],
       [11, 16, 10, 11, 11, 14,  9,  9, 13,  9, 11, 13, 12, 17, 11,  8],
       [10, 10, 13,  9, 18, 11, 13, 10, 13, 11,  7,

In [106]:
valid_predictions==1

array([False, False, False, ..., False, False, False], dtype=bool)